In [ ]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision as tv
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision import datasets
from rich import print
import matplotlib.pyplot as plt

In [ ]:
# Device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# Tranning data
train_data = datasets.FashionMNIST(
    root="FashionDataset",
    download=False,
    train=True,
    transform=ToTensor(),
    target_transform=None
)

# Test data
test_data = datasets.FashionMNIST(
    root="FashsionDataset",
    download=False,
    train=False,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
# Visualizing dataset
class_names = train_data.classes
image, title = train_data[1]

plt.imshow(image.squeeze(), cmap="gray")
plt.title(class_names[title])
plt.axis(False)
plt.show()

In [ ]:
torch.manual_seed(42)

fig = plt.figure(figsize=(9, 9))
rows, cols = 4, 4

for i in range(1, rows * cols + 1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(class_names[label])
    plt.axis(False)

plt.show()

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(batch_size=32,
                            dataset=train_data,
                            shuffle=True)

test_dataloader = DataLoader(batch_size=32,
                             dataset=test_data,
                             shuffle=False)

313